In [110]:
# Iniciar o selenium e exporta o arquivo

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Configurando as opções do Brave
chrome_options = Options()
chrome_options.binary_location = r"C:\Program Files\BraveSoftware\Brave-Browser\Application\brave.exe"

# Configurando o serviço do driver
servico = Service(ChromeDriverManager().install())

# Iniciando o navegador com as opções configuradas
navegador = webdriver.Chrome(service=servico, options=chrome_options)

# Carregar a tabela de produtos
tabela_produtos = pd.read_excel("buscas.xlsx")
display(tabela_produtos)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,Memória D35 Ddr4 16gb 3200mhz,rgb,150,320
1,"Gabinete Gamer Mancer Cv500, Mid T Lateral De ...",Cv500l,280,380


In [78]:
# Verificação de termos banidos e se tem todos os termos

def verificar_termos_banidos(lista_termos_banidos, nome):
    # analisar se ele não tem nenhum termo banido
    tem_termos_banidos = False
    for palavra in lista_termos_banidos:
        if palavra in nome:
            tem_termos_banidos = True
    return tem_termos_banidos

def verificar_tem_todos_termos_produto(lista_termos_nomes_produtos, nome):
    # analisar se ele tem todos os termos do nome do produto
    tem_todo_termos_produtos = True
    for palavra in lista_termos_nomes_produtos:
        if palavra not in nome:
            tem_todo_termos_produtos = False
    return tem_todo_termos_produtos

In [79]:
# mercado_livre
def mercado_livre(navegador, produto, termos_banidos, preco_minimo, preco_maximo):
    navegador.get("https://www.mercadolivre.com.br/")

    navegador.find_element('class name', 'nav-search-input').click()

    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_nomes_produtos = produto.split(" ")
    lista_de_ofertas_mercadolivre = []
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)

    navegador.find_element('class name', 'nav-search-input').send_keys(produto, Keys.ENTER)

    time.sleep(1.5)  # Espera a página carregar
    lista_resultados = navegador.find_elements('class name', 'ui-search-layout__item')
    print(f"Total de resultados encontrados: {len(lista_resultados)}")

    for resultado in lista_resultados:
        try:
            nome = resultado.find_element('class name', 'poly-component__title-wrapper').text
            nome = nome.lower()

            tem_termos_banidos = verificar_termos_banidos(lista_termos_banidos, nome)
            tem_todo_termos_produtos = verificar_tem_todos_termos_produto(lista_termos_nomes_produtos, nome)

            if not tem_termos_banidos and tem_todo_termos_produtos:
                preco = resultado.find_element('class name', 'andes-money-amount--cents-superscript').text
                preco = preco.replace("\n", "").replace("R$", "").replace(".", "").replace(",", ".").strip()
                preco = float(preco)

                if preco_minimo <= preco <= preco_maximo:
                    link_elemento = resultado.find_element('css selector', 'a.poly-component__title')
                    link = link_elemento.get_attribute('href')
                    lista_de_ofertas_mercadolivre.append((nome, preco, link))
        except Exception as e:
            print (f"Erro ao processar resultado: {e}")
    
    return lista_de_ofertas_mercadolivre



In [118]:
produto2 = "Memoria RAM 35d 16GB DDR4 3200MHz"

navegador.get("https://www.kabum.com.br/")

time.sleep(2)  # Aguarde um pouco mais para o site carregar

campo_pesquisa = navegador.find_element(By.ID, 'inputBusca')

# Define o valor do campo via JavaScript
navegador.execute_script("arguments[0].value = arguments[1];", campo_pesquisa, produto2)
time.sleep(0.2)

# Dispara o evento de submit do formulário de busca via JavaScript
navegador.execute_script("""
    arguments[0].form.dispatchEvent(new Event('submit', {bubbles: true}));
""", campo_pesquisa)

navegador.find_element(By.ID, 'inputBusca').send_keys(Keys.ENTER)

In [80]:
tabela_ofertas = pd.DataFrame()

for linha in tabela_produtos.index:
    produto = tabela_produtos.loc[linha, "Nome"]
    termos_banidos = tabela_produtos.loc[linha, "Termos banidos"]
    preco_minimo = tabela_produtos.loc[linha, "Preço mínimo"]
    preco_maximo = tabela_produtos.loc[linha, "Preço máximo"]

    ofertas_mercadolivre = mercado_livre(navegador, produto, termos_banidos, preco_minimo, preco_maximo)

    if ofertas_mercadolivre:
        df_ofertas = pd.DataFrame(ofertas_mercadolivre, columns=['Nome', 'Preço', 'Link'])
        df_ofertas['Produto'] = produto
        tabela_ofertas = pd.concat([tabela_ofertas, df_ofertas], ignore_index=True)

display(tabela_ofertas)

Total de resultados encontrados: 57
Total de resultados encontrados: 28


,Nome,Preço,Link,Produto
0,memória ram adata 3200mhz 16gb gammix d35 ddr4...,280.00,https://www.mercadolivre.com.br/memoria-ram-ad...,Memória D35 Ddr4 16gb 3200mhz
1,memória xpg gammix d35 ddr4 16gb 3200mhz branc...,309.51,https://produto.mercadolivre.com.br/MLB-391450...,Memória D35 Ddr4 16gb 3200mhz
2,memória ram adata 3200mhz 16gb gammix d35 ddr4...,317.00,https://produto.mercadolivre.com.br/MLB-361874...,Memória D35 Ddr4 16gb 3200mhz
3,memória xpg gammix d35 ddr4 16gb 3200mhz preto...,318.15,https://produto.mercadolivre.com.br/MLB-528790...,Memória D35 Ddr4 16gb 3200mhz
4,"memória ddr4 xpg gammix d35, 16gb, 3200mhz, white",305.90,https://produto.mercadolivre.com.br/MLB-400265...,Memória D35 Ddr4 16gb 3200mhz
5,memória ram adata 3200mhz 16gb gammix d35 ddr4...,311.00,https://produto.mercadolivre.com.br/MLB-408572...,Memória D35 Ddr4 16gb 3200mhz
6,memória ram adata 3200mhz 16gb gammix d35 ddr4...,293.00,https://produto.mercadolivre.com.br/MLB-408562...,Memória D35 Ddr4 16gb 3200mhz
